In [1]:
!pip install git+https://github.com/goolig/dsClass.git

  Cloning https://github.com/goolig/dsClass.git to /tmp/pip-req-build-4n9u7l5h
  Running command git clone -q https://github.com/goolig/dsClass.git /tmp/pip-req-build-4n9u7l5h
     |████████████████████████████████| 286kB 779kB/s eta 0:00:01
  Created wheel for dsClass: filename=dsClass-1.0.27-cp36-none-any.whl size=16372735 sha256=0334d3df972faf71d254e71a7aa0f3c90c7f629a75535c2ababb4c9d4a9bf661
  Stored in directory: /tmp/pip-ephem-wheel-cache-wbj_uway/wheels/c7/a9/03/023d8da4b4e004497715eeb1ea9b5d914ad425036bb4bc10b8
  Created wheel for pydotplus: filename=pydotplus-2.0.2-cp36-none-any.whl size=24566 sha256=6bd4469b8baaf3d408c903f8d579f6ef6721ae4d6f12ab9a55492a02a1fb0da6
  Stored in directory: /root/.cache/pip/wheels/35/7b/ab/66fb7b2ac1f6df87475b09dc48e707b6e0de80a6d8444e3628
Successfully built dsClass pydotplus


# BGU Data Science Course

In [2]:
# imports:
import pandas as pd
from dsClass.path_helper import *

### load the data:

In [3]:
theft_data_path = get_file_path('theft_data_bgu.csv')
theft_data = pd.read_csv(theft_data_path)
holiday_data_path = get_file_path('holidays_data_bgu.csv')
holidays_data_bgu = pd.read_csv(holiday_data_path)

In [4]:
# check the columns:
theft_data.columns

Index(['Date', 'District', 'ID', 'Case Number', 'Block', 'IUCR',
       'Primary Type', 'Description', 'Location Description', 'Arrest',
       'Domestic', 'Beat', 'Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Count'],
      dtype='object')

In [5]:
#Q1:

# consider only important columns:
# keep only the following columns in your dataframe- Date, District, Count(our label), Year

#### insert your code here:
theft_data = theft_data[['Date', 'District','Count', 'Year']]
print(theft_data.head())
####

theft_data.columns

         Date  District  Count  Year
0  2013-01-01      25.0   23.0  2013
1  2013-01-01      12.0   14.0  2013
2  2013-01-01       7.0   12.0  2013
3  2013-01-01       6.0   12.0  2013
4  2013-01-01       1.0   15.0  2013


Index(['Date', 'District', 'Count', 'Year'], dtype='object')

### Feature extraction:

In [6]:
#Q2:

# change the Date from string to timestamp, in theft_data & holidays_data_bgu:
# hint - use dt.datetime.strptime
import datetime as dt

#### insert your code here:
for row in theft_data['Date']:
    row = dt.datetime.strptime(row,"%Y-%m-%d")
    
for row in holidays_data_bgu['Date']:
    row = dt.datetime.strptime(row,"%Y-%m-%d")

print(theft_data.head())
print(holidays_data_bgu.head())
####


         Date  District  Count  Year
0  2013-01-01      25.0   23.0  2013
1  2013-01-01      12.0   14.0  2013
2  2013-01-01       7.0   12.0  2013
3  2013-01-01       6.0   12.0  2013
4  2013-01-01       1.0   15.0  2013
         Date  is_holiday
0  2012-01-02           1
1  2012-01-16           1
2  2012-02-20           1
3  2012-05-28           1
4  2012-07-04           1


In [7]:
# add time information:

#Q3:

# create day of week column:
# name the column Week_day, it will contain a number for each day of week: 
# Monday=0, ...., Sunday=6
#### insert your code here:
week_days = []
for date in theft_data['Date']:
    s = pd.date_range(date, date, freq='D').to_series()
    week_days.append(s.dt.dayofweek[0])
    
theft_data["Week_day"] = week_days

print(theft_data["Week_day"].describe())

####

#Q4:
# create season column:
# name the column Season, it will contain a string which tells us the season based on the Date: 
# hint - use the following package to indicate specific dates:
from datetime import date
# and write function that gets the date and returns the corresponding season
#### insert your code here:

def getSeason(month):
    mon = int(month[5:-3])
    if (mon in (12, 1, 2)):
        return "WINTER"
    elif(mon in (3, 4, 5)):
        return "SPRING"
    elif(mon in (6, 7, 8)):
        return "SUMMER"
    else:
        return "FALL"
    
season = []
for date in theft_data['Date']:
    season.append(getSeason(date))
    
theft_data["Season"] = season
print(theft_data.head())
print(theft_data["Season"].describe())

####

count    32142.000000
mean         3.000000
std          1.998319
min          0.000000
25%          1.000000
50%          3.000000
75%          5.000000
max          6.000000
Name: Week_day, dtype: float64
         Date  District  Count  Year  Week_day  Season
0  2013-01-01      25.0   23.0  2013         1  WINTER
1  2013-01-01      12.0   14.0  2013         1  WINTER
2  2013-01-01       7.0   12.0  2013         1  WINTER
3  2013-01-01       6.0   12.0  2013         1  WINTER
4  2013-01-01       1.0   15.0  2013         1  WINTER
count      32142
unique         4
top       SPRING
freq        8096
Name: Season, dtype: object


In [8]:
#Q5:

# add information from other sources
# holidays data
# create column is_holiday that will take 1 if its an holiday and 0 otherwise.

#### insert your code here:
theft_data = pd.merge(theft_data,holidays_data_bgu,how='left').fillna(0.0)
print(theft_data["is_holiday"].describe())
theft_data.head()
####

count    32142.000000
mean         0.027379
std          0.163186
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: is_holiday, dtype: float64


,Date,District,Count,Year,Week_day,Season,is_holiday
0,2013-01-01,25.0,23.0,2013,1,WINTER,1.0
1,2013-01-01,12.0,14.0,2013,1,WINTER,1.0
2,2013-01-01,7.0,12.0,2013,1,WINTER,1.0
3,2013-01-01,6.0,12.0,2013,1,WINTER,1.0
4,2013-01-01,1.0,15.0,2013,1,WINTER,1.0


In [9]:
# now you suppose to have the following columns: Date, is_holiday, district, count(our label), year, week_day, season
theft_data.columns

Index(['Date', 'District', 'Count', 'Year', 'Week_day', 'Season',
       'is_holiday'],
      dtype='object')

In [10]:
#Q6:
# create dummy variables:
# hint - use the pandas function 'get_dummies'
#### insert your code here:

#getDummies - mask of 0 or 1 for some property

cat_col=["Season","Week_day","District"]
theft_data = pd.get_dummies(data=theft_data,columns=cat_col)
print(theft_data.columns)
theft_data.head()
####

Index(['Date', 'Count', 'Year', 'is_holiday', 'Season_FALL', 'Season_SPRING',
       'Season_SUMMER', 'Season_WINTER', 'Week_day_0', 'Week_day_1',
       'Week_day_2', 'Week_day_3', 'Week_day_4', 'Week_day_5', 'Week_day_6',
       'District_1.0', 'District_2.0', 'District_3.0', 'District_4.0',
       'District_5.0', 'District_6.0', 'District_7.0', 'District_8.0',
       'District_9.0', 'District_10.0', 'District_11.0', 'District_12.0',
       'District_14.0', 'District_15.0', 'District_16.0', 'District_17.0',
       'District_18.0', 'District_19.0', 'District_20.0', 'District_22.0',
       'District_24.0', 'District_25.0'],
      dtype='object')


,Date,Count,Year,is_holiday,Season_FALL,Season_SPRING,Season_SUMMER,Season_WINTER,Week_day_0,Week_day_1,...,District_14.0,District_15.0,District_16.0,District_17.0,District_18.0,District_19.0,District_20.0,District_22.0,District_24.0,District_25.0
0,2013-01-01,23.0,2013,1.0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,2013-01-01,14.0,2013,1.0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2013-01-01,12.0,2013,1.0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2013-01-01,12.0,2013,1.0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2013-01-01,15.0,2013,1.0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# now you suppose to have the following columns: Date, is_holiday, count, year,
# dummies for season, week_day and district
theft_data.columns

Index(['Date', 'Count', 'Year', 'is_holiday', 'Season_FALL', 'Season_SPRING',
       'Season_SUMMER', 'Season_WINTER', 'Week_day_0', 'Week_day_1',
       'Week_day_2', 'Week_day_3', 'Week_day_4', 'Week_day_5', 'Week_day_6',
       'District_1.0', 'District_2.0', 'District_3.0', 'District_4.0',
       'District_5.0', 'District_6.0', 'District_7.0', 'District_8.0',
       'District_9.0', 'District_10.0', 'District_11.0', 'District_12.0',
       'District_14.0', 'District_15.0', 'District_16.0', 'District_17.0',
       'District_18.0', 'District_19.0', 'District_20.0', 'District_22.0',
       'District_24.0', 'District_25.0'],
      dtype='object')

In [12]:
#Q7:

# remove the original date columns: (we won't use it for modelling)
#### insert your code here:
theft_data = theft_data.drop('Date', axis=1)
theft_data.columns
####

Index(['Count', 'Year', 'is_holiday', 'Season_FALL', 'Season_SPRING',
       'Season_SUMMER', 'Season_WINTER', 'Week_day_0', 'Week_day_1',
       'Week_day_2', 'Week_day_3', 'Week_day_4', 'Week_day_5', 'Week_day_6',
       'District_1.0', 'District_2.0', 'District_3.0', 'District_4.0',
       'District_5.0', 'District_6.0', 'District_7.0', 'District_8.0',
       'District_9.0', 'District_10.0', 'District_11.0', 'District_12.0',
       'District_14.0', 'District_15.0', 'District_16.0', 'District_17.0',
       'District_18.0', 'District_19.0', 'District_20.0', 'District_22.0',
       'District_24.0', 'District_25.0'],
      dtype='object')

### Train-Test Splitting:

In [13]:
# choose years for train and test
train_start_year = 2014
train_end_year = 2015
test_year = 2016

In [14]:
# split the data into train/test
dataTrain = theft_data[(theft_data["Year"] >= train_start_year) & (theft_data["Year"] <= train_end_year)]
labelsTrain = dataTrain.Count
dataTrain = dataTrain.drop('Count', axis=1)


dataTest = theft_data[(theft_data["Year"] == test_year)]
labelsTest = dataTest.Count
dataTest = dataTest.drop('Count', axis=1)

# Remove unnecessary columns:
dataTrain = dataTrain.drop('Year', axis=1)
dataTest = dataTest.drop('Year', axis=1)

print("Train data shape: " , dataTrain.shape)
print("Test data shape: " , dataTest.shape)

Train data shape:  (16060, 34)
Test data shape:  (8052, 34)


In [15]:
# check for null values (should print 0)
print(theft_data.isnull().sum().sum())

0


### Modelling

In [16]:
from sklearn.linear_model import LinearRegression

mlModel = LinearRegression()

mlModel.fit(dataTrain, labelsTrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Evaluation

In [17]:
predTest = mlModel.predict(dataTest)

# print the Rsquare:
print("Test set R^2: ", mlModel.score(dataTest, labelsTest))

Test set R^2:  0.5649523112796103


### Question:

In [18]:
#Q8:
# train the model on years 2013-2015 and test it on 2016.

train_start_year = 2013
train_end_year = 2015
test_year = 2016

# split the data into train/test
dataTrain = theft_data[(theft_data["Year"] >= train_start_year) & (theft_data["Year"] <= train_end_year)]
labelsTrain = dataTrain.Count
dataTrain = dataTrain.drop('Count', axis=1)


dataTest = theft_data[(theft_data["Year"] == test_year)]
labelsTest = dataTest.Count
dataTest = dataTest.drop('Count', axis=1)

# Remove unnecessary columns:
dataTrain = dataTrain.drop('Year', axis=1)
dataTest = dataTest.drop('Year', axis=1)

mlModel = LinearRegression()

mlModel.fit(dataTrain, labelsTrain)

predTest = mlModel.predict(dataTest)

# print the Rsquare:
print("2013-2015 Train, 2016 Test R^2: ", mlModel.score(dataTest, labelsTest))




2013-2015 Train, 2016 Test R^2:  0.559567392606984


In [19]:
#how does the model results change?
#we can see that there is not significant difference in the R^2 between 2013-2015 train and test 2016 to 2014-2015 train and test 2016.

In [20]:
# try other combinations of train/test


train_start_year = 2013
train_end_year = 2014
test_year = 2016

# split the data into train/test
dataTrain = theft_data[(theft_data["Year"] >= train_start_year) & (theft_data["Year"] <= train_end_year)]
labelsTrain = dataTrain.Count
dataTrain = dataTrain.drop('Count', axis=1)


dataTest = theft_data[(theft_data["Year"] == test_year)]
labelsTest = dataTest.Count
dataTest = dataTest.drop('Count', axis=1)

# Remove unnecessary columns:
dataTrain = dataTrain.drop('Year', axis=1)
dataTest = dataTest.drop('Year', axis=1)

mlModel = LinearRegression()

mlModel.fit(dataTrain, labelsTrain)

predTest = mlModel.predict(dataTest)

# print the Rsquare:
print("2013-2014 Train, 2016 Test R^2: ", mlModel.score(dataTest, labelsTest))



train_start_year = 2013
train_end_year = 2013
test_year = 2016

# split the data into train/test
dataTrain = theft_data[(theft_data["Year"] >= train_start_year) & (theft_data["Year"] <= train_end_year)]
labelsTrain = dataTrain.Count
dataTrain = dataTrain.drop('Count', axis=1)


dataTest = theft_data[(theft_data["Year"] == test_year)]
labelsTest = dataTest.Count
dataTest = dataTest.drop('Count', axis=1)

# Remove unnecessary columns:
dataTrain = dataTrain.drop('Year', axis=1)
dataTest = dataTest.drop('Year', axis=1)

mlModel = LinearRegression()

mlModel.fit(dataTrain, labelsTrain)

predTest = mlModel.predict(dataTest)

# print the Rsquare:
print("2013-2013 Train, 2016 Test R^2: ", mlModel.score(dataTest, labelsTest))



2013-2014 Train, 2016 Test R^2:  0.5304093112431989
2013-2013 Train, 2016 Test R^2:  0.47591965435632544


In [21]:
# how does the model results change?
# we can see that there are differences in the R^2:
# 2013-2014 train and test 2016 much lower.
# 2013-2013 train and test 2016 even lower.

# R^2 get lower maybe because of the time we train the model, we start with large amount of data (2013-2015) and then small amount
# (only one year 2013-2013). 